In [ ]:
import cv2
import numpy as np
import pytesseract
import os

# C:\Program Files\Tesseract-OCR

per = 35
pixelThreshold=500

roi = [[(18, 198), (44, 226), 'box', 'pvt car'],
       [(132, 200), (162, 226), 'box', 'two wheeler'],
       [(452, 202), (482, 224), 'box', 'new'],
       [(144, 270), (708, 292), 'text', 'proposal no'],
       [(140, 304), (708, 324), 'text', 'quote no'],
       [(146, 336), (712, 356), 'box', 'inward no'],
       [(140, 372), (710, 392), 'text', 'inspection no'],
       [(770, 370), (996, 396), 'text', 'state'],
       [(1076, 370), (1390, 396), 'text', 'sp code'],
       [(822, 266), (1020, 298), 'text', 'rm code'],
       [(820, 306), (1024, 324), 'text', 'sec rm code'],
       [(824, 338), (1014, 358), 'text', 'receipt no'],
       [(1164, 268), (1388, 292), 'text', 'agreement code'],
       [(1162, 304), (1390, 322), 'text', 'agreement name'],
       [(1164, 340), (1388, 360), 'text', 'receipt date'],
       [(554, 202), (578, 226), 'box', 'renewal'],
       [(682, 198), (706, 224), 'box', 'rot over'],
       [(822, 200), (852, 224), 'box', 'used'],
       [(928, 202), (962, 224), 'box', 'endorsment'],
       [(1152, 142), (1180, 170), 'box', 'package'],
       [(1290, 144), (1318, 174), 'box', 'liability'],
       [(140, 410), (172, 434), 'box', 'urban'],
       [(232, 412), (264, 432), 'box', 'rural'],
       [(318, 412), (348, 430), 'box', 'social'],
       [(540, 406), (760, 432), 'text', 'gstin'],
       [(924, 410), (952, 434), 'box', 'agency'],
       [(1026, 410), (1056, 432), 'box', 'banca'],
       [(1128, 408), (1156, 432), 'box', 'corporait'],
       [(1312, 410), (1338, 434), 'box', 'direct'],[(112, 378), (226, 404), 'text', 'Title'], [(312, 382), (1392, 400), 'text', 'Name'], [(902, 348), (1392, 372), 'text', 'Policy Number'], [(114, 406), (144, 426), 'box', 'Male'], [(236, 410), (256, 426), 'box ', 'female'], [(654, 410), (880, 426), 'text', 'dob'], [(992, 412), (1396, 430), 'text', 'contct'], [(116, 434), (878, 448), 'text', 'emailid'], [(226, 462), (908, 478), 'text', 'occupation'], [(1050, 482), (1390, 500), 'text', 'pan no.']]

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

imgQ = cv2.imread('page1.png')
h,w,c = imgQ.shape
#imgQ = cv2.resize(imgQ,(w//3,h//3))

orb = cv2.ORB_create(10000)
kp1, des1 = orb.detectAndCompute(imgQ,None)
impKp1 = cv2.drawKeypoints(imgQ,kp1,None)
#cv2.imshow(x,imgQ)
path = 'forms'
myPicList = os.listdir(path)
print(myPicList)
for j,y in enumerate(myPicList):
    img = cv2.imread(path +"/"+y)
    #img = cv2.resize(img, (w, h ))
    # cv2.imshow(y, img)
    kp2, des2 = orb.detectAndCompute(img,None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.match(des2,des1)
    matches.sort(key= lambda x: x.distance)
    good = matches[:int(len(matches)*(per/100))]
    imgMatch = cv2.drawMatches(img,kp2,imgQ,kp1,good[:100],None,flags=2)

    #cv2.imshow(y, imgMatch)
    imgShow = img.copy()
    #cv2.imshow("ww",imgShow)
    imgMask = np.zeros_like(imgShow)
    
    myData = []

    print(f'################## Extracting Data from Form {j}  ##################')

    for x,r in enumerate(roi):

        cv2.rectangle(imgMask, (r[0][0],r[0][1]),(r[1][0],r[1][1]),(0,255,0),cv2.FILLED)
        imgShow = cv2.addWeighted(imgShow,0.99,imgMask,0.1,0)

        imgCrop = img[r[0][1]:r[1][1], r[0][0]:r[1][0]]
        #cv2.imshow(str(x), imgCrop)

        if r[2] == 'text':
            #print(f'{r[3]} :{pytesseract.image_to_string(imgCrop)}')
            print('{} :{}'.format(r[3],pytesseract.image_to_string(imgCrop)))
            myData.append(pytesseract.image_to_string(imgCrop))
            
        if r[2] =='box':
            imgGray = cv2.cvtColor(imgCrop,cv2.COLOR_BGR2GRAY)
            imgThresh = cv2.threshold(imgGray,170,255, cv2.THRESH_BINARY_INV)[1]
            totalPixels = cv2.countNonZero(imgThresh)
            if totalPixels>pixelThreshold: totalPixels =1;
            else: totalPixels=0
            print(f'{r[3]} :{totalPixels}')
            myData.append(totalPixels)
        #cv2.putText(imgShow,str(myData[x]),(r[0][0],r[0][1]),
                    #cv2.FONT_HERSHEY_PLAIN,2.5,(0,0,255),4)

    with open('DataOutput.csv','a+') as f:
        for data in myData:
            f.write((str(data)+','))
        f.write('\n')

    imgShow = cv2.resize(imgShow, (w , h ))
    print(myData)
    cv2.imshow(y+"2", imgShow)
    cv2.imwrite(y,imgShow)


cv2.imshow("KeyPointsQuery",impKp1)
cv2.imshow("Output",imgQ)
cv2.waitKey(0)
cv2.destroyAllWindows

['p-1.jpg', 'pp1.jpg']
################## Extracting Data from Form 0  ##################
pvt car :1
two wheeler :1
new :1
proposal no :MAN ADS PL
quote no :
inward no :1
inspection no :TNE HOVRTTAS cfatad Nalnwe arn minimiim ---;
state :SA Ce a ee Ld
sp code :
rm code :HUOW THO
sec rm code :
receipt no :
agreement code :
agreement name :
receipt date :
renewal :1
rot over :1
used :1
endorsment :1
package :1
liability :1
urban :1
rural :1
social :1
gstin :
agency :1
banca :1
corporait :1
direct :1
Title :TUN Y
Name :VIAITU UGIUW ale MINIMUM ragitiroamant tm Wow fiirriehkanl ho, - Men pene The
Policy Number :
Male :1
dob :
contct :
emailid :lIoeroee
occupation :
pan no. :
[1, 1, 1, 'MAN ADS PL', '', 1, 'TNE HOVRTTAS cfatad Nalnwe arn minimiim ---;', 'SA Ce a ee Ld', '', 'HUOW THO', '', '', '', '', '', 1, 1, 1, 1, 1, 1, 1, 1, 1, '', 1, 1, 1, 1, 'TUN Y', 'VIAITU UGIUW ale MINIMUM ragitiroamant tm Wow fiirriehkanl ho, - Men pene The', '', 1, '', '', 'lIoeroee', '', '']
################## E